## 继承MODULE类来构造模型

Module 类是 nn 模块⾥提供的⼀个模型构造类，是所有神经⽹络模块的基类，我们可以继承它来定义
我们想要的模型。

In [3]:
import torch
from torch import nn

class MLP(nn.Module):
    #声明带有模型参数的层，这里声明了两个全连接层
    def __init__(self,**kwargs):
        super(MLP,self).__init__(**kwargs)
        self.hidden=nn.Linear(784,256)
        self.act=nn.ReLU()
        self.output=nn.Linear(256,10)
    #定义模型的前向计算，即如何根据输入x计算返回所需要的模型输出
    def forward(self,x):
        a=self.act(self.hidden(x))
        return self.output(a)

In [4]:
X=torch.rand(2,784)
net=MLP()
print(net)
net(X)

MLP(
  (hidden): Linear(in_features=784, out_features=256, bias=True)
  (act): ReLU()
  (output): Linear(in_features=256, out_features=10, bias=True)
)


tensor([[ 0.1216,  0.1289,  0.1792, -0.2059, -0.1853, -0.2181,  0.1470,  0.1223,
         -0.0382,  0.0663],
        [ 0.1149,  0.0916,  0.0808, -0.2868, -0.2926, -0.1868, -0.0096,  0.1593,
         -0.1564,  0.0213]], grad_fn=<AddmmBackward>)

## MODULE的子类

### Sequential类

当模型的前向计算为简单串联各个层的计算时， Sequential 类可以通过更加简单的⽅式定义模型。这
正是 Sequential 类的⽬的：它可以接收⼀个⼦模块的有序字典（OrderedDict）或者⼀系列⼦模块作
为参数来逐⼀添加 Module 的实例，⽽模型的前向计算就是将这些实例按添加的顺序逐⼀计算。

In [5]:
class MySequential(nn.Module):
    from collections import OrderedDict
    def __init__(self,*args):
        super(MySequential,self).__init__()
        if len(args)==1 and isinstance(args[0],OrderedDict):
            for key,module in args[0].items():
                self.add_module(key,module)# add_module⽅法会将module添加进self._modules(⼀个OrderedDict)
        else:
            for idx,module in enumerate(args):
                self.add_module(str(idx),module)
                
    def forward(self,input):
        for module in self._modules.values():
            input=module(input)
        return input

In [6]:
net=MySequential(
    nn.Linear(784,256),
    nn.ReLU(),
    nn.Linear(256,10),
)
print(net)
net(X)

MySequential(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


tensor([[ 0.1146, -0.1560, -0.0271, -0.0986, -0.1116, -0.1496,  0.1060,  0.0225,
          0.0967,  0.0420],
        [ 0.0696, -0.1562, -0.0302,  0.0043, -0.0368, -0.0221,  0.3237,  0.0855,
          0.2227,  0.0649]], grad_fn=<AddmmBackward>)

### ModuleList类

ModuleList 接收⼀个⼦模块的列表作为输⼊，然后也可以类似List那样进⾏append和extend操作。

In [7]:
net=nn.ModuleList([nn.Linear(784,256),nn.ReLU()])
net.append(nn.Linear(256,10))
print(net[-1])
print(net)

Linear(in_features=256, out_features=10, bias=True)
ModuleList(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


### ModuleDict类

ModuleDict 接收⼀个⼦模块的字典作为输⼊, 然后也可以类似字典那样进⾏添加访问操作。

In [8]:
net=nn.ModuleDict({
    'linear':nn.Linear(784,256),
    'act':nn.ReLU(),
})
net['output']=nn.Linear(256,10)
print(net['linear'])
print(net.output)
print(net)

Linear(in_features=784, out_features=256, bias=True)
Linear(in_features=256, out_features=10, bias=True)
ModuleDict(
  (linear): Linear(in_features=784, out_features=256, bias=True)
  (act): ReLU()
  (output): Linear(in_features=256, out_features=10, bias=True)
)


## 构造复杂的模型

In [9]:
class FancyMLP(nn.Module):
    def __init__(self,**kwargs):
        super(FancyMLP,self).__init__(**kwargs)
        
        self.rand_weight=torch.rand((20,20),requires_grad=False)#不可训练参数（常数参数）
        self.linear=nn.Linear(20,20)
    
    def forward(self,x):
        x=self.linear(x)
        #使用创建的常数参数，以及nn.functional中的relu函数和mm函数
        x=nn.functional.relu(torch.mm(x,self.rand_weight)+1)
        
        #复用全连接。等价于两个全连接层共享参数
        x=self.linear(x)
        #控制流，这里我们需要调用item函数来返回标量进行比较
        while x.norm().item()>1:
            x/=2
        if x.norm().item()<0.8:
            x*=10
        return x.sum()

In [10]:
X=torch.rand(2,20)
net=FancyMLP()
print(net)
net(X)

FancyMLP(
  (linear): Linear(in_features=20, out_features=20, bias=True)
)


tensor(-1.8456, grad_fn=<SumBackward0>)

In [11]:
class NestMLP(nn.Module):
    def __init__(self,**kwargs):
        super(NestMLP,self).__init__(**kwargs)
        self.net=nn.Sequential(nn.Linear(40,30),nn.ReLU())
    def forward(self,x):
        return self.net(x)
net=nn.Sequential(NestMLP(),nn.Linear(30,20),FancyMLP())
X=torch.rand(2,40)
print(net)
net(X)

Sequential(
  (0): NestMLP(
    (net): Sequential(
      (0): Linear(in_features=40, out_features=30, bias=True)
      (1): ReLU()
    )
  )
  (1): Linear(in_features=30, out_features=20, bias=True)
  (2): FancyMLP(
    (linear): Linear(in_features=20, out_features=20, bias=True)
  )
)


tensor(11.4486, grad_fn=<SumBackward0>)